# Advanced Querying Mongo

Importing libraries and setting up connection

In [28]:
from pymongo import MongoClient
dbName="companies"
mongodbURL=f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db=client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [29]:
query={"name":{"$eq":"Babelgum"}}
list(db["companies"].find(query,{"name":1}))


[{'_id': ObjectId('52cdef7c4bab8bd675297da0'), 'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [42]:
from pymongo import ASCENDING, DESCENDING
#pongo solo empleados, nombre y limit 10 para que no te eternices
query={"number_of_employees":{"$gt":5000}}
list(db.companies.find(query,{"name":1,"number_of_employees":1}).sort([("number_of_employees",DESCENDING)]).limit(10))

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'number_of_employees': 405000},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'),
  'name': 'IBM',
  'number_of_employees': 388000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'),
  'name': 'Toyota',
  'number_of_employees': 320000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'_id': ObjectId('52cdef7d4bab8bd675298aa4'),
  'name': 'Samsung Electronics',
  'number_of_employees': 221726},
 {'_id': ObjectId('52cdef7d4bab8bd675298b99'),
  'name': 'Accenture',
  'number_of_employees': 205000},
 {'_id': ObjectId('52cdef7e4bab8bd67529a657'),
  'name': 'Tata Consultancy Services',
  'number_of_employees': 200300},
 {'_id': ObjectId('52cdef7e4bab8bd67529aa51'),
  'name': 'Flextronics International',
  'number_of_employees': 200000},
 {'

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [49]:
query={"founded_year":{"$gte": 2000, "$lte":2005}}
cur=list(db.companies.find(query,{"name":1,"founded_year":1}))
cur[:8]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_year': 2004},
 {'_id': ObjectId('52cdef7c4bab8bd675297d96'),
  'name': 'Gizmoz',
  'founded_year': 2003},
 {'_id': ObjectId('52cdef7c4bab8bd675297d95'),
  'name': 'StumbleUpon',
  'founded_year': 2002},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9a'),
  'name': 'Helio',
  'founded_year': 2005},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'founded_year': 2004}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [53]:
query={"ipo.valuation_amount":{"$gt":100000000}, "founded_year":{"$lte": 2010}}
cur=list(db.companies.find(query,{"name":1,"ipo":1}))
cur[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297de0'),
  'name': 'Yelp',
  'ipo': {'valuation_amount': 1300000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 3,
   'pub_day': 2,
   'stock_symbol': 'NYSE:YELP'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e0c'),
  'name': 'LinkedIn',
  'ipo': {'valuation_amount': 9310000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2011,
   'pub_month': 7,
   'pub_day': 20,
   'stock_symbol': 'NYSE:LNKD'}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e81

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [61]:
query={"number_of_employees":{"$gt":5000},"founded_year":{"$lte": 2005}}
cur=list(db.companies.find(query).sort([("number_of_employees", ASCENDING)]).limit(10))
#no te lo ejecuto porque es larguísimo

### 6. All the companies that don't include the `partners` field.

In [66]:
query={"partners":{"$exists":False}} 
list(db.companies.find(query))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [73]:
query={"category_code":{"$type":"null"}}
cur=list(db.companies.find(query))
cur[:2]
#es larga, sorry

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'),
  'name': 'Collective',
  'permalink': 'collective',
  'crunchbase_url': 'http://www.crunchbase.com/company/collective',
  'homepage_url': None,
  'blog_url': None,
  'blog_feed_url': None,
  'twitter_username': None,
  'category_code': None,
  'number_of_employees': None,
  'founded_year': None,
  'founded_month': None,
  'founded_day': None,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': None,
  'alias_list': None,
  'email_address': None,
  'phone_number': None,
  'description': None,
  'created_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'updated_at': 'Thu Sep 26 13:15:02 UTC 2013',
  'overview': None,
  'image': None,
  'products': [],
  'relationships': [],
  'competitions': [],
  'providerships': [],
  'total_money_raised': '$0',
  'funding_rounds': [],
  'investments': [],
  'acquisition': None,
  'acquisitions': [],
  'offices': [],
  'milestones': [],
  'ipo': N

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [76]:
query={"number_of_employees":{"$gte": 100, "$lte":1000}}
cur=list(db.companies.find(query,{"name":1,"number_of_employees":1}))
cur[:4]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8b'),
  'name': 'AdventNet',
  'number_of_employees': 600},
 {'_id': ObjectId('52cdef7c4bab8bd675297da8'),
  'name': 'OpenX',
  'number_of_employees': 305},
 {'_id': ObjectId('52cdef7c4bab8bd675297da7'),
  'name': 'AddThis',
  'number_of_employees': 120},
 {'_id': ObjectId('52cdef7c4bab8bd675297db5'),
  'name': 'LifeLock',
  'number_of_employees': 644}]

### 9. Order all the companies by their IPO price in a descending order.

In [83]:
query={}
cur=list(db.companies.find(query).sort([("ipo.valuation_amount",DESCENDING)]).limit(4))
#Ras es eterno no te lo ejecuto

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [88]:
query={}
cur=list(db.companies.find(query,{"name":1,"number_of_employees":1}).sort([("number_of_employees",DESCENDING)]).limit(10))
cur[:5]

[{'_id': ObjectId('52cdef7d4bab8bd67529941a'),
  'name': 'Siemens',
  'number_of_employees': 405000},
 {'_id': ObjectId('52cdef7c4bab8bd67529856a'),
  'name': 'IBM',
  'number_of_employees': 388000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d33'),
  'name': 'Toyota',
  'number_of_employees': 320000},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'number_of_employees': 300000},
 {'_id': ObjectId('52cdef7e4bab8bd67529b0fe'),
  'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [95]:
query={"founded_month":{"$gt":6}}
cur=list(db.companies.find(query,{"name":1,"founded_month":1}).limit(1000))
cur[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
  'name': 'Wetpaint',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8f'),
  'name': 'Omnidrive',
  'founded_month': 11},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8c'),
  'name': 'Zoho',
  'founded_month': 9},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'founded_month': 10},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9d'),
  'name': 'Joost',
  'founded_month': 10}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [102]:
query={"founded_year":{"$lt":2000},"acquisition.price_amount":{"$gt":10000}}
cur=list(db.companies.find(query,{"name":1,"acquisition":1}).limit(5))
cur[:3]

[{'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'acquisition': {'price_amount': 625000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/',
   'source_description': 'http://www.techcrunch.com/2007/07/09/google-acquires-postini-for-625-million/',
   'acquired_year': 2007,
   'acquired_month': 7,
   'acquired_day': 9,
   'acquiring_company': {'name': 'Google', 'permalink': 'google'}}},
 {'_id': ObjectId('52cdef7c4bab8bd675297deb'),
  'name': 'SideStep',
  'acquisition': {'price_amount': 180000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://www.xconomy.com/2007/12/21/kayak-sidestep-will-travel-together-in-rare-east-buys-west-acquisition/',
   'source_description': 'xconomy',
   'acquired_year': 2007,
   'acquired_month': 12,
   'acquired_day': 1,
   'acquiring_company': {'name': 'KAYAK', 'permalink': 'kayak'}}},
 {'_id': Obj

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [104]:
query={"founded_year":{"$gt":2010}}
cur=list(db.companies.find(query,{"name":1,"acquisition":1}).sort([("acquisition.price_amount",DESCENDING)]).limit(5))
cur

[{'_id': ObjectId('52cdef7c4bab8bd675297ea7'),
  'name': 'Mokitown',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297f78'),
  'name': 'CircleUp',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297f94'),
  'name': 'PeekYou',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297fb1'),
  'name': 'headr',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'acquisition': None}]

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [105]:
query={}
list(db.companies.find(query,{"name":1,"founded_year":1}).sort([("founded_year",DESCENDING)]).limit(5))


[{'_id': ObjectId('52cdef7c4bab8bd675297fec'),
  'name': 'Fixya',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529801f'),
  'name': 'Wamba',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd6752982d4'),
  'name': 'Advaliant',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7c4bab8bd67529830a'),
  'name': 'Fluc',
  'founded_year': 2013},
 {'_id': ObjectId('52cdef7d4bab8bd675298ea7'),
  'name': 'iBazar',
  'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [109]:
query={"founded_day":{"$lte":7}}
list(db.companies.find(query,{"name":1,"founded_day":1,"acquisition.price_amount":1}).sort([("acquisition.price_amount",DESCENDING)]).limit(8))

[{'_id': ObjectId('52cdef7d4bab8bd6752989a1'),
  'name': 'Netscape',
  'founded_day': 4,
  'acquisition': {'price_amount': 4200000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297e89'),
  'name': 'PayPal',
  'founded_day': 1,
  'acquisition': {'price_amount': 1500000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297efe'),
  'name': 'Zappos',
  'founded_day': 1,
  'acquisition': {'price_amount': 1200000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297f0c'),
  'name': 'Alibaba',
  'founded_day': 1,
  'acquisition': {'price_amount': 1000000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675297d90'),
  'name': 'Postini',
  'founded_day': 2,
  'acquisition': {'price_amount': 625000000}},
 {'_id': ObjectId('52cdef7c4bab8bd67529831c'),
  'name': 'Danger',
  'founded_day': 1,
  'acquisition': {'price_amount': 500000000}},
 {'_id': ObjectId('52cdef7c4bab8bd675298651'),
  'name': 'Clearwell Systems',
  'founded_day': 6,
  'acquisition': {'price_amount': 410000000}},
 {'_id': ObjectId('52cdef7d4bab8bd6752989b8

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [116]:
query={"category_code":"web","number_of_employees":{"$gt":4000}}
cur=list(db.companies.find(query,{"name":1,"category":1,"number_of_employees":1,"category_code":1}).sort([("number_of_employees",DESCENDING)]))
cur[:6]

[{'_id': ObjectId('52cdef7c4bab8bd6752982a8'),
  'name': 'Experian',
  'category_code': 'web',
  'number_of_employees': 15500},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'category_code': 'web',
  'number_of_employees': 15000},
 {'_id': ObjectId('52cdef7c4bab8bd675297da3'),
  'name': 'Yahoo!',
  'category_code': 'web',
  'number_of_employees': 13600},
 {'_id': ObjectId('52cdef7c4bab8bd675297fcb'),
  'name': 'Rakuten',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7c4bab8bd67529834c'),
  'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'_id': ObjectId('52cdef7d4bab8bd675299d5d'),
  'name': 'Groupon',
  'category_code': 'web',
  'number_of_employees': 10000}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [120]:
query={"acquisition.price_currency_code":"EUR","acquisition.price_amount":{"$gt":10000000}}
cur=list(db.companies.find(query,{"name":1,"acquisition.price_currency_code":1,"acquisition.price_amount":1}))
cur[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297f02'),
  'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7d4bab8bd675298bf3'),
  'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7d4bab8bd675298f47'),
  'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a536'),
  'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'_id': ObjectId('52cdef7e4bab8bd67529a729'),
  'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [124]:
query={"acquisitions.acquired_month":{"$lte":3}}
cur=list(db.companies.find(query,{"name":1,"acquisition":1}).limit(10))
cur[:5]

[{'_id': ObjectId('52cdef7c4bab8bd675297d8e'),
  'name': 'Facebook',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9e'),
  'name': 'CBS',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297d94'),
  'name': 'Twitter',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9f'),
  'name': 'Viacom',
  'acquisition': None},
 {'_id': ObjectId('52cdef7c4bab8bd675297d9b'),
  'name': 'eBay',
  'acquisition': None}]

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code